In [ ]:
%pip install matplotlib seaborn scikit-learn lightgbm -q

^C


In [2]:
# 라이브러리 임포트

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.ensemble import RandomForestClassifier

# LightGBM
import lightgbm as lgb

# 시각화 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

import warnings
warnings.filterwarnings("ignore")


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# BigQuery 연결 설정

client = bigquery.Client()

PROJECT_ID = 'sesac-sd-bigquery-478605'  
DATASET = 'project'        

print(f"프로젝트: {PROJECT_ID}")
print(f"데이터셋: {DATASET}")

프로젝트: sesac-sd-bigquery-478605
데이터셋: project


In [4]:
# 분석용 데이터 추출

client = bigquery.Client()

query = f"""
WITH user_behavior AS (
    SELECT 
        u.user_id,
        u.mbti,
        u.mbti_e_i,
        u.mbti_s_n,
        u.mbti_t_f,
        u.mbti_j_p,
        u.customer_grade,
        u.is_churned,
        
        -- 세션 행동
        COUNT(DISTINCT s.session_id) as total_sessions,
        AVG(s.session_duration_sec) as avg_session_duration,
        SUM(s.page_views) as total_page_views,
        SUM(s.cart_additions) as total_cart_additions,
        SUM(CASE WHEN s.is_converted THEN 1 ELSE 0 END) as converted_sessions,
        
        -- 디바이스
        MAX(CASE WHEN s.device = 'mobile' THEN 1 ELSE 0 END) as uses_mobile,
        MAX(CASE WHEN s.device = 'desktop' THEN 1 ELSE 0 END) as uses_desktop
        
    FROM `{PROJECT_ID}.{DATASET}.users` u
    LEFT JOIN `{PROJECT_ID}.{DATASET}.sessions` s ON u.user_id = s.user_id
    WHERE u.is_bot = FALSE AND (s.is_bot = FALSE OR s.is_bot IS NULL)
    GROUP BY u.user_id, u.mbti, u.mbti_e_i, u.mbti_s_n, u.mbti_t_f, u.mbti_j_p, u.customer_grade, u.is_churned
)
SELECT * FROM user_behavior
"""

df = client.query(query).to_dataframe()
print(f"데이터 shape: {df.shape}")
df.head()

데이터 shape: (4954, 15)


,user_id,mbti,mbti_e_i,mbti_s_n,mbti_t_f,mbti_j_p,customer_grade,is_churned,total_sessions,avg_session_duration,total_page_views,total_cart_additions,converted_sessions,uses_mobile,uses_desktop
0,USER_000517,ENFJ,E,N,F,J,GOLD,False,125,918.800000,854,306,7,1,1
1,USER_002852,ENFJ,E,N,F,J,NORMAL,False,128,910.156250,786,240,5,1,1
2,USER_004422,ENFJ,E,N,F,J,NORMAL,False,122,875.754098,810,262,15,1,1
3,USER_004437,ENFJ,E,N,F,J,NORMAL,False,133,892.180451,909,287,7,1,1
4,USER_004484,ENFJ,E,N,F,J,NORMAL,False,125,895.296000,852,266,5,1,1


In [7]:
# 데이터 확인

print("기본 정보")
print(df.info())

print("\n결측치 확인")
print(df.isnull().sum())

print("\n타겟 변수 분포 (is_churned)")
print(df['is_churned'].value_counts())
print(df['is_churned'].value_counts(normalize=True).round(3))

기본 정보
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4954 entries, 0 to 4953
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   user_id               4954 non-null   object 
 1   mbti                  4954 non-null   object 
 2   mbti_e_i              4954 non-null   object 
 3   mbti_s_n              4954 non-null   object 
 4   mbti_t_f              4954 non-null   object 
 5   mbti_j_p              4954 non-null   object 
 6   customer_grade        4954 non-null   object 
 7   is_churned            4954 non-null   boolean
 8   total_sessions        4954 non-null   Int64  
 9   avg_session_duration  4954 non-null   float64
 10  total_page_views      4954 non-null   Int64  
 11  total_cart_additions  4954 non-null   Int64  
 12  converted_sessions    4954 non-null   Int64  
 13  uses_mobile           4954 non-null   Int64  
 14  uses_desktop          4954 non-null   Int64  
dtypes: Int64(6), bo

In [11]:
# Feature 준비 - MBTI 제외 버전

# 타겟 변수
target = 'is_churned'

# MBTI 제외한 행동 변수만 사용
behavior_features = [
    'total_sessions', 
    'avg_session_duration', 
    'total_page_views', 
    'total_cart_additions', 
    'converted_sessions',
    'uses_mobile',
    'uses_desktop',
    'customer_grade'
]

df_model = df.copy()

# 인코딩
grade_map = {'NORMAL': 0, 'GOLD': 1, 'VIP': 2}
df_model['customer_grade'] = df_model['customer_grade'].map(grade_map)

# Feature / Target 분리
X_behavior = df_model[behavior_features]
y = df_model[target].astype(int)

print("MBTI 제외")
print(X_behavior.columns.tolist())
print(f"\nShape: {X_behavior.shape}")

MBTI 제외
['total_sessions', 'avg_session_duration', 'total_page_views', 'total_cart_additions', 'converted_sessions', 'uses_mobile', 'uses_desktop', 'customer_grade']

Shape: (4954, 8)


In [10]:
# Feature 준비 - MBTI 포함 버전

# MBTI 축 인코딩
df_model['mbti_e_i'] = df_model['mbti_e_i'].map({'E': 1, 'I': 0})
df_model['mbti_s_n'] = df_model['mbti_s_n'].map({'S': 1, 'N': 0})
df_model['mbti_t_f'] = df_model['mbti_t_f'].map({'T': 1, 'F': 0})
df_model['mbti_j_p'] = df_model['mbti_j_p'].map({'J': 1, 'P': 0})

# MBTI 포함 Feature
mbti_features = behavior_features + ['mbti_e_i', 'mbti_s_n', 'mbti_t_f', 'mbti_j_p']

X_with_mbti = df_model[mbti_features]

print("MBTI 포함")
print(X_with_mbti.columns.tolist())
print(f"\nShape: {X_with_mbti.shape}")

MBTI 포함
['total_sessions', 'avg_session_duration', 'total_page_views', 'total_cart_additions', 'converted_sessions', 'uses_mobile', 'uses_desktop', 'customer_grade', 'mbti_e_i', 'mbti_s_n', 'mbti_t_f', 'mbti_j_p']

Shape: (4954, 12)
